In [11]:
import os
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as soups
from time import sleep

def search_selenium(search_name, search_path, keyword):
    search_url = "https://www.google.com/search?q=" + str(search_name) + "&hl=ko&tbm=isch"
    chrome_options = webdriver.ChromeOptions()
    
    # ChromeDriver 경로 설정
    browser = webdriver.Chrome(service=webdriver.chrome.service.Service(executable_path='C:/Program Files/Google/Chrome/Application/chromedriver.exe'), options=chrome_options)
    
    browser.get(search_url)

    try:
        element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.ID, "islrg"))
        )
    except:
        print("Timeout waiting for the page to load.")
        browser.quit()
        return

    last_height = browser.execute_script("return document.body.scrollHeight")

#     print("로드된 이미지 개수 : ", len(browser.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")))

    search_limit = int(input("원하는 이미지 수집 개수 : "))
    image_count = 0
    while image_count < search_limit:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(2)
        images = browser.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
        for image in images:
            try:
                image.click()
                sleep(1)
                # This part selects the larger image if available.
                try:
                    actual_image = browser.find_element(By.CSS_SELECTOR, ".n3VNCb")
                except:
                    actual_image = image
                
                # 페이지 제목
                soup = soups(browser.page_source, "html.parser")
                title = soup.find("div", class_="tvh9oe BIB1wf").text
                
                # 특정 단어가 포함된 페이지면 크롤링 / 띄어쓰기 해서 (3D 아기) 라고 치면 각각 찾음
                words = keyword.split()
                for word in words:
                    if word in title:
                        image_filename = os.path.join(search_path, f"{image_count}.jpg")
                        image_count += 1
                    if image_count >= search_limit:
                        break
            except:
                continue
        last_height = browser.execute_script("return document.body.scrollHeight")
        if browser.execute_script("return document.body.scrollHeight") == last_height:
            break
    browser.quit()

In [12]:
search_name = input("검색하고 싶은 키워드 : ")
keyword_test = input('특정단어 :')
crawling_path = input("저장할 폴더명 입력 : ")
search_path = "C:/Users/johan/Desktop/" + crawling_path # 폴더 경로 설정
try:
    # 중복되는 폴더 명이 없다면 생성
    if not os.path.exists(search_path):
        os.makedirs(search_path)
    # 중복된다면 문구 출력 후 프로그램 종료
    else:
        print('이전에 같은 [검색어, 이미지 수]로 다운로드한 폴더가 존재합니다.')
        sys.exit(0)
except OSError:
    print('os error')
    sys.exit(0)

search_selenium(search_name, search_path, keyword_test)

검색하고 싶은 키워드 : 3D fetus
특정단어 :ultrasound fetal
저장할 폴더명 입력 : 태아데이터
원하는 이미지 수집 개수 : 100
